# Messages and Prompting in LangChain

**Purpose:**  
Explain the message abstraction in LangChain, how it represents conversational state, and how different message types affect model behavior.

## Messages

Messages are the canonical representation of conversational state in LangChain.  
They encode both the content exchanged with the model and the structural context required for reasoning, tool use, and tracing.

Each message consists of:
- **Role** — Semantic function in the conversation (system, human, AI, tool).
- **Content** — The payload (text, images, audio, structured data, or empty for tool calls).
- **Metadata** — Optional identifiers, token usage, timestamps, tool call IDs, and provider-specific fields.

Using message objects instead of raw strings enables:
- Deterministic replay of conversations,
- Tool call orchestration,
- Metadata inspection and tracing,
- Cross-provider portability.


In [2]:
import os
from langchain.chat_models import init_chat_model

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model = init_chat_model("groq:qwen/qwen3-32b")
model.invoke("Please tell me what artificial intelligence is.")


AIMessage(content='<think>\nOkay, the user is asking for an explanation of artificial intelligence. Let me start by breaking down the term. AI is a broad field, so I need to make sure I cover the basics without getting too technical. \n\nFirst, I should define AI in simple terms. Maybe something like the simulation of human intelligence in machines. Then, mention the key areas it covers—machine learning, natural language processing, problem-solving. Wait, the user might not know those terms, so I should explain them briefly.\n\nI need to differentiate between different types of AI. There\'s narrow AI, which does specific tasks like voice assistants, and general AI, which is more human-like but not yet developed. Including examples would help make it clearer. Like, Siri or self-driving cars as examples of narrow AI.\n\nI should also talk about how AI works, maybe touch on machine learning and deep learning as subsets. Explain that AI systems learn from data, not just follow strict rules

## Text prompts

A text prompt is a raw string passed directly to the model.  
It is stateless and does not preserve conversational history.

In [3]:
model.invoke("What is LangChain?")

AIMessage(content='<think>\nOkay, the user asked "What is LangChain?" and I need to explain it in Chinese. Let me start by recalling what I know about LangChain. It\'s a framework for building applications with large language models. The user might be a developer or someone interested in AI, looking to understand how to use LLMs in practical projects.\n\nFirst, I should define LangChain clearly. Maybe mention it\'s a framework designed to work with LLMs, helping developers integrate them into applications. I should highlight the key components: agents, prompts, memory, and chains. These are the building blocks that allow for more complex and dynamic interactions with the models.\n\nI need to explain each part briefly. For example, agents are like the decision-makers that determine when to call a model or a specific action. Prompts are the templates for generating inputs to the model. Memory helps maintain context across interactions, which is crucial for tasks that require multiple ste

Use text prompts when:
- The request is independent and standalone,
- No prior context is required,
- You want minimal orchestration overhead.

## Message prompts

A message prompt is an explicit list of structured message objects.  
This form is used when conversation state, roles, tools, or metadata matter.

### Message types

- **SystemMessage** — Defines global instructions, behavioral constraints, and role framing.
- **HumanMessage** — Represents user input.
- **AIMessage** — Represents model output (including text, tool calls, and metadata).
- **ToolMessage** — Represents the result of executing a tool call.

In [4]:
from langchain.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage("You are a poetry expert."),
    HumanMessage("Write a poem about artificial intelligence.")
]

response = model.invoke(messages)
response.content


'<think>\nOkay, the user wants me to write a poem about artificial intelligence. Let me start by brainstorming some key themes and imagery related to AI. I need to capture both its potential and its complexities. Maybe start with the birth of AI, using metaphors like circuits or code. Personification could work here, giving AI human-like qualities.\n\nI should touch on different aspects: learning, evolution, ethical concerns, and the relationship between humans and AI. Balance the wonder of technology with caution. Words like "algorithms," "neural networks," "data streams" come to mind. Maybe use contrasts between cold machinery and warmth of human emotion.\n\nStructure-wise, maybe use quatrains with a consistent rhyme scheme. Let me think of some rhyming pairs. Also, include some alliteration or assonance for rhythm. Need to convey progression over time, from creation to future possibilities. End with a reflection on coexistence and responsibility. Avoid clichés, make it fresh. Let me

### System messages

System messages prime the model with instructions that persist across the interaction.
They influence tone, role, output format, and reasoning constraints.

In [5]:
system_msg = SystemMessage("You are a helpful coding assistant.")

messages = [
    system_msg,
    HumanMessage("How do I create a REST API?")
]

response = model.invoke(messages)
print(response.content)


<think>
Okay, the user is asking how to create a REST API. Let me start by recalling what a REST API is. REST stands for Representational State Transfer, and it's a set of architectural constraints for building web services. So, the main goal is to allow communication between clients and servers using standard HTTP methods like GET, POST, PUT, DELETE, etc.

First, I need to outline the general steps. The user might not have a specific framework in mind, so maybe I should mention that the approach can vary depending on the programming language and framework they choose. But since they might be looking for a concrete example, I should pick a popular one. Flask for Python is a good choice because it's simple and widely used. Alternatively, Express.js for Node.js or Django for Python could also work, but Flask might be easier for a basic example.

Let me start by listing the general steps. They need to design the API endpoints, define the resources, set up the server, handle HTTP methods, 

In [6]:
# Rich system message with explicit behavioral constraints
system_msg = SystemMessage("""
You are a senior Python developer with expertise in web frameworks.
Always provide code examples and explain your reasoning.
Be concise but thorough.
""")

messages = [
    system_msg,
    HumanMessage("How do I create a REST API?")
]

response = model.invoke(messages)
print(response.content)


<think>
Okay, the user is asking how to create a REST API in Python using web frameworks. Let me break this down step by step.

First, I need to decide which framework to use. The most common ones are Flask and FastAPI. Flask is a good choice for beginners because it's simple and has a lot of community support. FastAPI is more modern and offers better performance, especially for async applications. Since the user is asking for a basic example, maybe start with Flask to keep it straightforward.

Next, setting up the project structure. They'll need to create a virtual environment to manage dependencies. That's a standard practice, so I should include instructions on creating and activating a virtual environment.

Then, installing Flask. The command would be 'pip install Flask'. After that, setting up the basic app. The simplest example is creating an 'app.py' file with a route that returns a JSON response. Using the Flask-RESTful extension could be helpful here because it simplifies crea

## Message metadata

Metadata fields allow tracking, tracing, and multi-user scenarios without affecting the semantic content.

In [7]:
from langchain.messages import HumanMessage

human_msg = HumanMessage(
    content="Hello!",
    name="alice",
    id="msg_123"
)

response = model.invoke([human_msg])
response


AIMessage(content='<think>\nOkay, the user said "Hello!" so I should respond in a friendly way. I need to make sure my response is welcoming and encourages them to ask questions or share what\'s on their mind. Let me keep it simple and open-ended. Maybe something like, "Hello! How can I assist you today?" That should work. I should also be ready to adapt my tone if they want something more formal or casual. Let me check if there\'s anything else I need to consider. No, that\'s good. Let\'s go with that.\n</think>\n\nHello! How can I assist you today? Feel free to ask any question or share what\'s on your mind—I\'m here to help! 😊', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 145, 'prompt_tokens': 10, 'total_tokens': 155, 'completion_time': 0.290140448, 'completion_tokens_details': None, 'prompt_time': 0.000386243, 'prompt_tokens_details': None, 'queue_time': 0.07581299, 'total_time': 0.290526691}, 'model_name': 'qwen/qwen3-32b', 'system_fingerprint': '

In [8]:
from langchain.messages import AIMessage, SystemMessage, HumanMessage

# Manually inserting an AI message into history (e.g., replay, correction, or simulation)
ai_msg = AIMessage("I'd be happy to help you with that.")

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("Can you help me?"),
    ai_msg,
    HumanMessage("Great. What's 2 + 2?")
]

response = model.invoke(messages)
print(response.content)
response.usage_metadata


<think>
Okay, the user asked for help, and I offered. Now they're asking "What's 2 + 2?" So I need to figure out how to respond. Let me start by recalling that 2 + 2 is a basic arithmetic question. The answer is 4, right? But maybe they want more than just the answer. Should I explain it step by step?

Wait, the user might be testing if I can handle simple math. They might be a student needing help with homework or just someone curious. Maybe they want to see if the assistant is reliable. I should confirm the answer clearly. Let me make sure there's no trick here. In standard arithmetic, 2 + 2 is definitely 4. No, there's no context where that's different unless they're using a different base, but the question doesn't mention that. So base 10 is safe.

Is there a need to explain addition basics? Probably not, but maybe a quick explanation would help. Like, adding two numbers where each is two, so combining gives four. But the user might just want the answer quickly. Let me check the pr

{'input_tokens': 54, 'output_tokens': 332, 'total_tokens': 386}

## Tool messages

When a model emits a tool call, the call itself is stored in an AIMessage.
The result of executing the tool must be returned as a ToolMessage referencing the original call ID.

In [9]:
from langchain.messages import AIMessage, ToolMessage, HumanMessage

ai_message = AIMessage(
    content=[],
    tool_calls=[{
        "name": "get_weather",
        "args": {"location": "San Francisco"},
        "id": "call_123"
    }]
)

tool_message = ToolMessage(
    content="Sunny, 72°F",
    tool_call_id="call_123"
)

messages = [
    HumanMessage("What's the weather in San Francisco?"),
    ai_message,
    tool_message,
]

response = model.invoke(messages)
tool_message, response


(ToolMessage(content='Sunny, 72°F', tool_call_id='call_123'),
 AIMessage(content="<think>\nOkay, the user asked for the weather in San Francisco. I used the get_weather tool with the location set to San Francisco. The response came back as sunny with 72°F. Now I need to present this information clearly. Let me start by confirming the location so the user knows it's accurate. Then mention the current conditions: sunny and the temperature. Maybe add a friendly note about it being a nice day to enjoy outdoors. Keep it simple and conversational. Check for any typos or formatting issues. Make sure the temperature is in the right unit (Fahrenheit here). Alright, that should cover it.\n</think>\n\nThe current weather in San Francisco is **sunny** with a temperature of **72°F**. It looks like a nice day to enjoy outdoor activities! ☀️", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 165, 'prompt_tokens': 57, 'total_tokens': 222, 'completion_time': 0.29003416, 'co

## Summary

- Messages are the structural representation of conversational state.
- Roles encode intent and control model behavior.
- System messages define global constraints.
- Metadata enables tracing, replay, and orchestration.
- Tool messages close the loop between model reasoning and external execution.